In [5]:
import sys
sys.path.append('./code')

import matrix_elements as me
import numpy as np
import copy

In [24]:
s1 = A.states[0]

In [90]:
print basis[0]
print basis[1]

|H, j=1, m=-1, w=-1>
|H, j=1, m=0, w=-1>


In [145]:
Stark = ElectricDipoleOperator(electronic_matrix_elements={('H', 0, 'H'):1})
Stark.matrix_element(basis[0], basis[1])

array([-0.35355339-0.j        ,  0.00000000-0.35355339j,  0.00000000-0.j        ])

In [187]:
class SphericalTensorOperator(object):
    
    operator = 'generic spherical tensor operator'
    rank = None
    
    def __init__(self, electronic_matrix_elements=None):
        self.spherical_vectors = {+1 : np.array([-1, -1j, 0], dtype='complex128')/np.sqrt(2),
                                   0 : np.array([ 0,   0, 1], dtype='complex128'),
                                  -1 : np.array([+1, -1j, 0], dtype='complex128')/np.sqrt(2)}
        self.electronic_matrix_elements = electronic_matrix_elements
    
    def to_spherical_vector(self, vector):
        vector = np.array(vector, dtype='complex128')
        s_vector = {}
        for k, v in self.spherical_vectors.items():
            s_vector[k] = np.dot(np.conj(v), vector)
        return s_vector
        
    def evaluate_matrix_element(self, state0=None, state1=None, 
                                initialize=0, function=None):
        """
        Vector is a cartesian 3d vector input. If this is None, the output
        is a vector matrix element. If it is not None, the output is 
        a scalar.
        """
        value = initialize
        # assume that state0 and state1 are expressed in Hunds Case (a)
        for a0, s0 in zip(state0.amplitudes, state0.states):
            for a1, s1 in zip(state1.amplitudes, state1.states):
                for q in range(-self.rank, self.rank + 1):
                    Q = s0.w - s1.w
                    e_trans1 = (s1.T, np.abs(Q), s0.T)
                    e_trans2 = (s0.T, np.abs(Q), s1.T)
                    c_1 = e_trans1 in self.electronic_matrix_elements.keys()
                    c_2 = e_trans2 in self.electronic_matrix_elements.keys()

                    if (c_1 or c_2):
                        if c_1:
                            M = self.electronic_matrix_elements[e_trans1]
                        elif c_2:
                            M = self.electronic_matrix_elements[e_trans2]

                        value += (np.conj(a0) * a1) * function(s0, s1, q, Q) * M
        return value
    
    def matrix_element(self, state0, state1):
        raise AttributeError("this is a virtual function to be overwritten")
        pass
    
    def _generate_hamiltonian(self, basis, initialize = 0):
        H = initialize
        for i in range(len(basis)):
            for j in range(i, len(basis)):
                si = basis[i]
                sj = basis[j]
                value = self.matrix_element(si, sj)
                H[i, j] = value
                if i != j:
                    H[j, i] = np.conj(value)
        return H
    
    
class ElectricDipoleOperator(SphericalTensorOperator):
    
    operator = 'rank 1 Electric Dipole Operator'
    rank = 1
        
    def matrix_element(self, state0=None, state1=None, vector=None):
        """
        Vector is a cartesian 3d vector input. If this is None, the output
        is a vector matrix element. If it is not None, the output is 
        a scalar.
        """
        value = np.array([0, 0, 0], dtype='complex128')
        
        function = lambda s0, s1, q, Q :(np.complex128(
                                         me.lab_to_mol(s0.j, s0.m, s0.w,
                                                       self.rank, q,  Q,
                                                       s1.j, s1.m, s1.w)) *
                                         self.spherical_vectors[q])
        
        value = self.evaluate_matrix_element(state0, state1,
                                             initialize = value,
                                             function = function)

        value = np.conj(value)
        if vector is not None:
            vector = np.array(vector, dtype='complex128')
            value = np.dot(value, vector)
        return value
    
    def generate_hamiltonian(self, basis):
        H = np.zeros((len(basis), len(basis), 3), dtype='complex128')
        H = self._generate_hamiltonian(basis, initialize=H)
        hamiltonian = lambda E : np.dot(H, np.array(E, dtype='complex128'))
        return hamiltonian
    

class MagneticDipoleOperator(SphericalTensorOperator):
    
    operator = 'rank 1 Magnetic Dipole Operator'
    rank = 1
        
    def matrix_element(self, state0=None, state1=None, vector=None):
        """
        Vector is a cartesian 3d vector input. If this is None, the output
        is a vector matrix element. If it is not None, the output is 
        a scalar.
        """
        value = np.array([0, 0, 0], dtype='complex128')
        
        function = lambda s0, s1, q, Q :(
                                         np.complex128(
                                         me.lab_to_mol(s0.j, s0.m, s0.w,
                                                       self.rank, q,  Q,
                                                       s1.j, s1.m, s1.w)) *
                                         self.spherical_vectors[q])
        
        value = self.evaluate_matrix_element(state0, state1,
                                             initialize = value,
                                             function = function)

        value = np.conj(value)
        if vector is not None:
            vector = np.array(vector, dtype='complex128')
            value = np.dot(value, vector)
        return value
    
    def generate_hamiltonian(self, basis):
        H = np.zeros((len(basis), len(basis), 3), dtype='complex128')
        H = self._generate_hamiltonian(basis, initialize=H)
        hamiltonian = lambda E : np.dot(H, np.array(E, dtype='complex128'))
        return hamiltonian

    
class HundsCaseABasisState(object):
    
    def __init__(self,T=None, j=None, m=None, w=None):
        if j is None or m is None or w is None:
            raise IOError('j, m, and w must be defined.')
        self.j = j
        self.m = m
        self.w = w
        self.T = T
    
    def __str__(self):
        output =  ( 'j=' + str(self.j) + ', '
                  + 'm=' + str(self.m) + ', '
                  + 'w=' + str(self.w))
        
        if self.T is not None:
            output = (str(self.T) + ', ' + output)
        
        output = '|' + output + '>'
        return output
        
    
    def __repr__(self):
        return str(self)
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)

    
def hunds_case_a_basis(T=None, j=None, m=None, w=None):
    
    states = []
    
    if j is None or w is None:
        raise IOError('j and w must be defined')
    if not(isinstance(j,(list, tuple))):
        j = [j]
    if not(isinstance(w,(list, tuple))):
        w = [w]
    if not(isinstance(T,(list, tuple))):
        T = [T]
    if m is not None and not(isinstance(m,(list, tuple))):
        m = [m]
        
    for T_inst in T:
        for j_inst in j:
            for w_inst in w:
                if m is None:
                    m_temp = range(-j_inst, j_inst + 1)
                else:
                    m_temp = m
                    
                for m_inst in m_temp:
                    
                    basis_state = HundsCaseABasisState(T=T_inst, j=j_inst,
                                                       m=m_inst, w=w_inst)
                    state = State(1,basis_state)
                    states.append(state)
                    
                    
    if len(states) == 1:
        states = states[0]
    
    return states


class State(object):
    
    def __init__(self, amplitudes, states):
        
        if not(isinstance(amplitudes,(list,tuple)) 
               or amplitudes.__class__.__name__ == 'ndarray'):
            amplitudes = [amplitudes]
            
        if not(isinstance(states, (list,tuple))):
            states     = [states]
            
        if len(amplitudes) != len(states):
            raise IOError("""the state object must be provided equal
                          length amplitude and states lists""")
            
        self.amplitudes = np.array(amplitudes)
        self.states     = list(states)
        
    def __len__(self):
        return len(self.states)
        
    def __str__(self):
        output = []
        for a, s in zip(self.amplitudes, self.states):
            output.append(' + ')
            if a != 1:
                output.append(str(a))
            output.append(str(s))
            
        output.pop(0)
        output = ''.join(output)
        return output
    
    def normalize(self):
        self.amplitudes = self.amplitudes / self.norm()
        
    def norm(self):
        return np.sqrt(np.sum(self.amplitudes * np.conj(self.amplitudes)))
    
    def t(self):
        return State(np.conj(self.amplitudes),self.states)
    
    def __repr__(self):
        return str(self)
    
    def __mul__(self, other):
        if isinstance(other, self.__class__):
            if other.states == self.states:
                return sum(np.conj(other.amplitudes) * self.amplitudes)
            
            if len(self.states) >= len(other.states):
                large = self.t()
                small = other
            else:
                large = other
                small = self.t()
                
            total = 0
            for s, a in zip(small.states, list(small.amplitudes)):
                if s in large.states:
                    ind = large.states.index(s)
                    total += a * large.amplitudes[ind]  
            return total
        
        else:
            return State(self.amplitudes * other, self.states)
    
    def __rmul__(self, other):
        if isinstance(other, self.__class__):
            return np.conj(self.__mul__(other))
        else:
            return self.__mul__(other)
    
    def __add__(self, other):
        if not(isinstance(other, self.__class__)):
            raise TypeError('only state objects can be added together')
            
        new_states = copy.deepcopy(self.states)
        new_amplitudes = list(copy.deepcopy(self.amplitudes))
        
        for s, a in zip(other.states,list(other.amplitudes)):
            if s in new_states:
                ind = new_states.index(s)
                new_amplitudes[ind] += a
            else:
                new_states.append(s)
                new_amplitudes.append(a)
                
        return State(new_amplitudes, new_states)
    
    def __sub__(self, other):
        return self.__add__((-1) * other)
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            if len(self.states) != len(other.states):
                return False
            else:
                for s, a in zip(self.states, list(self.amplitudes)):
                    if s in other.states:
                        ind = other.states.index(s)
                        if other.amplitudes[ind] != a:
                            return False
                    else:
                        return False
                return True
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)

In [7]:
a = hunds_case_a_basis(T='H',j=1,m=1,w=1)
b = hunds_case_a_basis(T='H',j=2,m=1,w=1)
c = hunds_case_a_basis(T='H',j=2,m=1,w=1)

In [111]:
basis = hunds_case_a_basis(T=['H'], j=[1], w=[-1, 1])
basis

[|H, j=1, m=-1, w=-1>,
 |H, j=1, m=0, w=-1>,
 |H, j=1, m=1, w=-1>,
 |H, j=1, m=-1, w=1>,
 |H, j=1, m=0, w=1>,
 |H, j=1, m=1, w=1>]

In [192]:
electric_dipole_moments = {
    ('H', 0, 'H'): 1, 
    ('H', 0, 'C'): 1,
    ('C', 0, 'C'): 1
}

magnetic_dipole_moments = {
    ('H', 0, 'H'): 1, 
    ('H', 0, 'C'): 1,
    ('C', 0, 'C'): 1
}

H_stark = ElectricDipoleOperator(electric_dipole_moments).generate_hamiltonian(basis)
H_zeeman= MagneticDipoleOperator(magnetic_dipole_moments).generate_hamiltonian(basis)

In [193]:
H_stark([1,0,1])

array([[ 0.50000000+0.j, -0.35355339+0.j,  0.00000000+0.j,  0.00000000+0.j,
         0.00000000+0.j,  0.00000000+0.j],
       [-0.35355339+0.j,  0.00000000+0.j, -0.35355339+0.j,  0.00000000+0.j,
         0.00000000+0.j,  0.00000000+0.j],
       [ 0.00000000+0.j, -0.35355339+0.j, -0.50000000+0.j,  0.00000000+0.j,
         0.00000000+0.j,  0.00000000+0.j],
       [ 0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j, -0.50000000+0.j,
         0.35355339+0.j,  0.00000000+0.j],
       [ 0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,  0.35355339+0.j,
         0.00000000+0.j,  0.35355339+0.j],
       [ 0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,  0.00000000+0.j,
         0.35355339+0.j,  0.50000000+0.j]])

In [148]:
print basis[0]
print basis[1]
print basis[2]

|H, j=1, m=-1, w=-1>
|H, j=1, m=0, w=-1>
|H, j=1, m=1, w=-1>


In [150]:
print h_stark.operator.electronic_matrix_elements
print h_stark.operator.matrix_element(basis[0], basis[1])

{('H', 1, 'H'): 1, ('C', 1, 'C'): 1, ('H', 1, 'C'): 1}


array([ 0.-0.j,  0.-0.j,  0.-0.j])

In [117]:
H_stark([0,0,1])

array([[ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j]])

In [19]:
si = basis[0].states[0]

In [17]:
si = basis[0]
si = basis[0]

In [14]:
me.lab_to_mol?

In [13]:
print A
print B
print A * B
print B * A

|H, j=1, m=1, w=1> + (1+1j)|H, j=2, m=1, w=1>
|H, j=1, m=1, w=1> + 1j|H, j=2, m=1, w=1>
(2-1j)
(2+1j)


In [10]:
states = [a, b, c]
A = states[0] + (1 + 1j) * states[1]
B = states[0] + 1j * states[1]

#A.normalize()
#B.normalize()

print(A * B)
print(B * A)

(2-1j)
(2+1j)


In [ ]:
me.lab_to_mol?

In [2]:
me.lab_to_mol(1, 0, 1, 
              1,-1, 0,
              1, 1, 1)/np.sqrt(2)

0.353553390593274

In [ ]:
(me.lab_to_mol(1, 0, 1, 
              1, 1, 0,
              1, 1, 1)
-
me.lab_to_mol(1, 0, 1, 
              1, 1, 0,
              1, 1, 1))